In [2]:
!pip install --upgrade gdown
import gdown

# https://drive.google.com/file/d/1CPdZPpT6rBKZx83VxCsG9kgJumpVOf3F/view?usp=drive_link
file_id = "1CPdZPpT6rBKZx83VxCsG9kgJumpVOf3F"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="[F2]sumarizovani_tekstovi (1).zip", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1CPdZPpT6rBKZx83VxCsG9kgJumpVOf3F
From (redirected): https://drive.google.com/uc?id=1CPdZPpT6rBKZx83VxCsG9kgJumpVOf3F&confirm=t&uuid=d6471e2c-4bca-4a08-8155-1a32a4b92942
To: /content/[F2]sumarizovani_tekstovi (1).zip
100%|██████████| 485M/485M [00:06<00:00, 69.9MB/s]


'[F2]sumarizovani_tekstovi (1).zip'

In [3]:
import zipfile

with zipfile.ZipFile("[F2]sumarizovani_tekstovi (1).zip", 'r') as zip_ref:
    zip_ref.extractall("[F2]sumarizovani_tekstovi")

In [4]:
!python -c "import nltk; nltk.download('punkt')"
!pip install sumy transformers datasets torch pandas

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [5]:
# -*- coding: utf-8 -*-
"""
# Faza 3: Izračunavanje evaluacijskih metrika (Compression, Coverage, Density)
"""

import os
import json
import csv
import pandas as pd
import re
import numpy as np
from collections import Counter
from google.colab import drive

# Povezivanje Google Drive-a
drive.mount('/content/drive')

# Bosanske stop riječi - koristićemo za izračunavanje gustoće (density)
STOP_RIJECI = [
    'i', 'u', 'je', 'da', 'se', 'na', 'su', 'za', 'od', 'što', 'kako', 'ali', 'a', 'ili',
    'to', 'nije', 'sam', 'mi', 'ti', 'on', 'ona', 'ono', 'oni', 'one', 'smo', 'ste', 'sa',
    'jer', 'po', 'kad', 'kao', 'do', 'iz', 'bi', 'te', 'tako', 'pri', 'nakon', 'već', 'o',
    'kroz', 'još', 'samo', 'sve', 'ako', 'mogu', 'može', 'biti', 'bio', 'bila', 'bilo',
    'bili', 'bile', 'će', 'ću', 'ćemo', 'ćete', 'neće', 'ne', 'niti', 'pa', 'dok',
    'prije', 'ovaj', 'ova', 'ovo', 'ovi', 'ove', 'taj', 'ta', 'to', 'ti', 'te', 'svoj',
    'svoja', 'svoje', 'svoji', 'svoje', 'moj', 'moja', 'moje', 'moji', 'moje'
]

# Funkcija za pronalaženje ispravne putanje
def find_correct_path(folder_name):
    print(f"Tražim ispravnu putanju za {folder_name}...")

    # Provjerite nekoliko mogućih lokacija
    potential_paths = [
        f'/content/drive/MyDrive/{folder_name}',
        f'/content/drive/My Drive/{folder_name}',
        f'/content/drive/Shared drives/{folder_name}'
    ]

    # Provjera svih putanja
    for path in potential_paths:
        print(f"Provjeravam: {path}")
        if os.path.exists(path):
            print(f"Pronađena putanja: {path}")
            return path

    # Ako ne možemo naći tačnu lokaciju, tražimo unos od korisnika
    print("\nNe mogu pronaći folder. Evo sadržaja vašeg drive-a:")
    root_contents = os.listdir('/content/drive')
    print(f"Root sadržaj: {root_contents}")

    if 'MyDrive' in root_contents or 'My Drive' in root_contents:
        mydrive_path = '/content/drive/MyDrive' if 'MyDrive' in root_contents else '/content/drive/My Drive'
        print(f"\nSadržaj vašeg MyDrive-a:")
        try:
            mydrive_contents = os.listdir(mydrive_path)
            for item in mydrive_contents:
                print(f"  - {item}")
        except Exception as e:
            print(f"Ne mogu pristupiti: {e}")

    # Tražimo unos od korisnika
    user_path = input(f"\nUnesite tačnu putanju do vašeg {folder_name} foldera (kopirajte putanju): ")
    if os.path.exists(user_path):
        return user_path
    else:
        print(f"Putanja {user_path} ne postoji. Koristiću standardnu putanju.")
        return f'/content/drive/MyDrive/{folder_name}'

# Funkcija za podjelu teksta na riječi (za izračunavanje metrika)
def podijeli_na_rijeci(text):
    # Prevođenje u mala slova i uklanjanje interpunkcije
    text = text.lower()
    words = re.findall(r'\b\w+\b', text)
    return words

# Funkcija za izračunavanje metrika
def izracunaj_metrike(originalni_tekst, sazetak):
    if not originalni_tekst or not sazetak:
        return {
            "Compression": 0,
            "Coverage": 0,
            "Density": 0,
            "Compression_bin": "none",
            "Coverage_bin": "none",
            "Density_bin": "none"
        }

    # Dobijanje riječi (bez stop riječi za gustoću)
    orig_rijeci = podijeli_na_rijeci(originalni_tekst)
    sazetak_rijeci = podijeli_na_rijeci(sazetak)

    # Izračunavanje riječi bez stop-riječi (za gustoću)
    orig_rijeci_bez_stop = [w for w in orig_rijeci if w not in STOP_RIJECI and len(w) > 2]
    sazetak_rijeci_bez_stop = [w for w in sazetak_rijeci if w not in STOP_RIJECI and len(w) > 2]

    # Brojanje jedinstvenih riječi bez stop-riječi
    orig_unique = set(orig_rijeci_bez_stop)
    sazetak_unique = set(sazetak_rijeci_bez_stop)

    # Metrike
    # 1. Compression: odnos dužine sažetka prema dužini originalnog teksta
    compression = (len(sazetak_rijeci) / len(orig_rijeci)) * 100 if len(orig_rijeci) > 0 else 0

    # 2. Coverage: procenat važnih riječi originalnog teksta koje su u sažetku
    coverage = len(sazetak_unique.intersection(orig_unique)) / len(orig_unique) if len(orig_unique) > 0 else 0

    # 3. Density: odnos važnih riječi prema ukupnoj dužini sažetka
    density = (len(sazetak_unique) / len(sazetak_rijeci)) * 100 if len(sazetak_rijeci) > 0 else 0

    # Određivanje kategorija (biniranje)
    compression_bin = "low" if compression < 20 else ("medium" if compression < 50 else "high")
    coverage_bin = "low" if coverage < 0.3 else ("medium" if coverage < 0.7 else "high")
    density_bin = "abstractive" if density < 25 else ("mixed" if density < 35 else "extractive")

    return {
        "Compression": round(compression, 10),
        "Coverage": round(coverage, 10),
        "Density": round(density, 10),
        "Compression_bin": compression_bin,
        "Coverage_bin": coverage_bin,
        "Density_bin": density_bin
    }

# Funkcija za procesiranje JSON fajla
def procesiraj_json_metrike(ulazni_fajl, izlazni_fajl):
    try:
        # Učitavanje JSON
        with open(ulazni_fajl, 'r', encoding='utf-8') as f:
            podaci = json.load(f)

        # Provjera da li je podaci lista (array) ili pojedinačni objekt
        if isinstance(podaci, list):
            # Ako je lista, procesiramo svaki objekt u listi
            for item in podaci:
                if 'Text' in item and ('ExtractiveSum' in item or 'AbstractiveSum' in item):
                    # Izračunavanje metrika za ekstraktivnu sumarizaciju
                    if 'ExtractiveSum' in item and item['ExtractiveSum']:
                        metrike_eks = izracunaj_metrike(item['Text'], item['ExtractiveSum'])
                        # Dodavanje prefiksa 'Extractive_' metrikama
                        for key, value in metrike_eks.items():
                            item[f'Extractive_{key}'] = value

                    # Izračunavanje metrika za abstraktivnu sumarizaciju
                    if 'AbstractiveSum' in item and item['AbstractiveSum']:
                        metrike_abs = izracunaj_metrike(item['Text'], item['AbstractiveSum'])
                        # Dodavanje prefiksa 'Abstractive_' metrikama
                        for key, value in metrike_abs.items():
                            item[f'Abstractive_{key}'] = value
        else:
            # Pojedinačni objekat
            if 'Text' in podaci:
                # Izračunavanje metrika za ekstraktivnu sumarizaciju
                if 'ExtractiveSum' in podaci and podaci['ExtractiveSum']:
                    metrike_eks = izracunaj_metrike(podaci['Text'], podaci['ExtractiveSum'])
                    # Dodavanje prefiksa 'Extractive_' metrikama
                    for key, value in metrike_eks.items():
                        podaci[f'Extractive_{key}'] = value

                # Izračunavanje metrika za abstraktivnu sumarizaciju
                if 'AbstractiveSum' in podaci and podaci['AbstractiveSum']:
                    metrike_abs = izracunaj_metrike(podaci['Text'], podaci['AbstractiveSum'])
                    # Dodavanje prefiksa 'Abstractive_' metrikama
                    for key, value in metrike_abs.items():
                        podaci[f'Abstractive_{key}'] = value

        # Zapis novog JSON-a
        os.makedirs(os.path.dirname(izlazni_fajl), exist_ok=True)
        with open(izlazni_fajl, 'w', encoding='utf-8') as f:
            json.dump(podaci, f, ensure_ascii=False, indent=4)

        return True
    except Exception as e:
        print(f"Greška pri obradi JSON fajla {ulazni_fajl}: {e}")
        return False

# Funkcija za procesiranje CSV fajla
def procesiraj_csv_metrike(ulazni_fajl, izlazni_fajl):
    try:
        # Čitanje CSV-a u pandas DataFrame
        df = pd.read_csv(ulazni_fajl, encoding='utf-8')

        # Provjera da li postoje potrebne kolone
        if 'Text' in df.columns:
            # Izračunavanje metrika za ekstraktivnu sumarizaciju
            if 'ExtractiveSum' in df.columns:
                metrike_df = df.apply(lambda row: izracunaj_metrike(row['Text'], row['ExtractiveSum'])
                                     if pd.notna(row['Text']) and pd.notna(row['ExtractiveSum'])
                                     else {"Compression": 0, "Coverage": 0, "Density": 0,
                                           "Compression_bin": "none", "Coverage_bin": "none", "Density_bin": "none"},
                                     axis=1)

                # Dodavanje kolona sa ekstraktivnim metrikama
                for key in ["Compression", "Coverage", "Density", "Compression_bin", "Coverage_bin", "Density_bin"]:
                    df[f'Extractive_{key}'] = metrike_df.apply(lambda x: x[key])

            # Izračunavanje metrika za abstraktivnu sumarizaciju
            if 'AbstractiveSum' in df.columns:
                metrike_df = df.apply(lambda row: izracunaj_metrike(row['Text'], row['AbstractiveSum'])
                                     if pd.notna(row['Text']) and pd.notna(row['AbstractiveSum'])
                                     else {"Compression": 0, "Coverage": 0, "Density": 0,
                                           "Compression_bin": "none", "Coverage_bin": "none", "Density_bin": "none"},
                                     axis=1)

                # Dodavanje kolona sa abstraktivnim metrikama
                for key in ["Compression", "Coverage", "Density", "Compression_bin", "Coverage_bin", "Density_bin"]:
                    df[f'Abstractive_{key}'] = metrike_df.apply(lambda x: x[key])

        # Zapis novog CSV-a
        os.makedirs(os.path.dirname(izlazni_fajl), exist_ok=True)
        df.to_csv(izlazni_fajl, index=False, encoding='utf-8')

        return True
    except Exception as e:
        print(f"Greška pri obradi CSV fajla {ulazni_fajl}: {e}")

        try:
            # Alternativni pristup ako pandas ne radi
            print("Pokušavam alternativni pristup za obradu CSV fajla...")
            redovi = []
            zaglavlja = []

            # Čitanje CSV-a ručno
            with open(ulazni_fajl, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                zaglavlja = next(reader)  # Prva linija su zaglavlja

                # Dodavanje novih polja za metrike
                for prefix in ['Extractive_', 'Abstractive_']:
                    for metriku in ["Compression", "Coverage", "Density", "Compression_bin", "Coverage_bin", "Density_bin"]:
                        nova_kolona = f"{prefix}{metriku}"
                        if nova_kolona not in zaglavlja:
                            zaglavlja.append(nova_kolona)

                # Indeksi potrebnih kolona
                try:
                    tekst_indeks = zaglavlja.index('Text')
                    eks_indeks = zaglavlja.index('ExtractiveSum') if 'ExtractiveSum' in zaglavlja else -1
                    abs_indeks = zaglavlja.index('AbstractiveSum') if 'AbstractiveSum' in zaglavlja else -1
                except ValueError:
                    print("CSV fajl ne sadrži potrebne kolone.")
                    return False

                # Obrada redova
                for red in reader:
                    # Ako nemamo dovoljno kolona, proširimo red
                    while len(red) < len(zaglavlja) - 12:  # -12 za nove kolone (6 za svaki tip sumarizacije)
                        red.append("")

                    # Izračunavanje metrika ako imamo tekst i sažetke
                    if tekst_indeks >= 0 and tekst_indeks < len(red) and red[tekst_indeks]:
                        tekst = red[tekst_indeks]

                        # Ekstraktivna sumarizacija
                        if eks_indeks >= 0 and eks_indeks < len(red) and red[eks_indeks]:
                            metrike_eks = izracunaj_metrike(tekst, red[eks_indeks])
                            for key, value in metrike_eks.items():
                                red.append(str(value))
                        else:
                            # Dodajemo prazne vrijednosti za metrike
                            for _ in range(6):
                                red.append("")

                        # Abstraktivna sumarizacija
                        if abs_indeks >= 0 and abs_indeks < len(red) and red[abs_indeks]:
                            metrike_abs = izracunaj_metrike(tekst, red[abs_indeks])
                            for key, value in metrike_abs.items():
                                red.append(str(value))
                        else:
                            # Dodajemo prazne vrijednosti za metrike
                            for _ in range(6):
                                red.append("")
                    else:
                        # Dodajemo prazne vrijednosti za sve metrike
                        for _ in range(12):
                            red.append("")

                    redovi.append(red)

            # Zapis novog CSV-a
            os.makedirs(os.path.dirname(izlazni_fajl), exist_ok=True)
            with open(izlazni_fajl, 'w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(zaglavlja)
                writer.writerows(redovi)

            return True
        except Exception as e2:
            print(f"Greška pri alternativnom pristupu za CSV: {e2}")
            return False

# Funkcija za obradu svih fajlova
def obradi_sve_fajlove(ulazni_folder, izlazni_folder):
    # Pronađimo podfolder buka.ba
    buka_folder = os.path.join(ulazni_folder, "buka.ba")

    if not os.path.exists(buka_folder):
        print(f"GREŠKA: Folder {buka_folder} ne postoji!")

        # Pokušajmo pronaći tačnu strukturu
        print("Tražim buka.ba folder...")

        # Ispisujemo sadržaj ulaznog foldera za dijagnostiku
        print(f"Sadržaj {ulazni_folder}:")
        try:
            for item in os.listdir(ulazni_folder):
                print(f"  - {item}")
                item_path = os.path.join(ulazni_folder, item)
                if os.path.isdir(item_path):
                    print(f"    Podfolderi u {item}:")
                    try:
                        for subitem in os.listdir(item_path):
                            print(f"      - {subitem}")
                    except Exception as e:
                        print(f"      Greška pri čitanju: {e}")
        except Exception as e:
            print(f"Greška pri čitanju direktorija: {e}")

        # Tražimo alternativne putanje
        alternative = [
            os.path.join(ulazni_folder, "buka_ba"),
            ulazni_folder  # Možda je glavni folder već buka.ba?
        ]

        for alt_path in alternative:
            if os.path.exists(alt_path):
                print(f"Pronađena alternativna putanja: {alt_path}")
                buka_folder = alt_path
                break

    # Kreirajmo izlazni folder
    izlazni_buka_folder = os.path.join(izlazni_folder, "buka.ba")
    os.makedirs(izlazni_buka_folder, exist_ok=True)

    processed_files = 0
    errors = 0

    # Prolazak kroz podfoldere
    print(f"Obrađujem fajlove iz {buka_folder}...")

    try:
        for item in os.listdir(buka_folder):
            item_path = os.path.join(buka_folder, item)

            if os.path.isdir(item_path):
                print(f"Obrada direktorija: {item}")

                # Kreiranje odgovarajućeg izlaznog podfoldera
                izlazni_podfolder = os.path.join(izlazni_buka_folder, item)
                os.makedirs(izlazni_podfolder, exist_ok=True)

                # Obrada fajlova u podfolderu
                for filename in os.listdir(item_path):
                    ulazni_fajl = os.path.join(item_path, filename)
                    izlazni_fajl = os.path.join(izlazni_podfolder, filename)

                    if filename.endswith('.json'):
                        print(f"  Obrada JSON: {filename}")
                        if procesiraj_json_metrike(ulazni_fajl, izlazni_fajl):
                            processed_files += 1
                        else:
                            errors += 1

                    elif filename.endswith('.csv'):
                        print(f"  Obrada CSV: {filename}")
                        if procesiraj_csv_metrike(ulazni_fajl, izlazni_fajl):
                            processed_files += 1
                        else:
                            errors += 1

            elif os.path.isfile(item_path):
                # Ako su fajlovi direktno u buka.ba folderu
                filename = item
                ulazni_fajl = item_path
                izlazni_fajl = os.path.join(izlazni_buka_folder, filename)

                if filename.endswith('.json'):
                    print(f"Obrada JSON: {filename}")
                    if procesiraj_json_metrike(ulazni_fajl, izlazni_fajl):
                        processed_files += 1
                    else:
                        errors += 1

                elif filename.endswith('.csv'):
                    print(f"Obrada CSV: {filename}")
                    if procesiraj_csv_metrike(ulazni_fajl, izlazni_fajl):
                        processed_files += 1
                    else:
                        errors += 1

    except Exception as e:
        print(f"Globalna greška pri obradi: {e}")

    return processed_files, errors

# Glavni program
if __name__ == "__main__":
    print("\n" + "=" * 50)
    print("FAZA 3: IZRAČUNAVANJE EVALUACIJSKIH METRIKA")
    print("=" * 50)

    # Pronalaženje ispravne putanje do ulaznih fajlova (iz Faze 2)
    INPUT_FOLDER = find_correct_path("[F2]SumarizovaniTekstovi")
    OUTPUT_FOLDER = os.path.join('/content/drive/MyDrive', "[F3]EvaluacijaTekstova")

    print(f"Ulazni folder: {INPUT_FOLDER}")
    print(f"Izlazni folder: {OUTPUT_FOLDER}")
    print("=" * 50)

    # Kreiranje izlaznog foldera
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f"Kreiran izlazni folder: {OUTPUT_FOLDER}")

    # Obrada svih fajlova
    processed, errors = obradi_sve_fajlove(INPUT_FOLDER, OUTPUT_FOLDER)

    print("\n" + "=" * 50)
    print(f"Obrada završena!")
    print(f"Uspješno obrađeno: {processed} fajlova")
    print(f"Greške: {errors}")
    print(f"Rezultati su sačuvani u folderu: {OUTPUT_FOLDER}")
    print("=" * 50)

Mounted at /content/drive

FAZA 3: IZRAČUNAVANJE EVALUACIJSKIH METRIKA
Tražim ispravnu putanju za [F2]SumarizovaniTekstovi...
Provjeravam: /content/drive/MyDrive/[F2]SumarizovaniTekstovi
Pronađena putanja: /content/drive/MyDrive/[F2]SumarizovaniTekstovi
Ulazni folder: /content/drive/MyDrive/[F2]SumarizovaniTekstovi
Izlazni folder: /content/drive/MyDrive/[F3]EvaluacijaTekstova
Kreiran izlazni folder: /content/drive/MyDrive/[F3]EvaluacijaTekstova
Obrađujem fajlove iz /content/drive/MyDrive/[F2]SumarizovaniTekstovi/buka.ba...
Obrada direktorija: Sport
  Obrada JSON: SportSviClanci.json
  Obrada CSV: SportSviClanci.csv
Obrada direktorija: TV
  Obrada JSON: TVSviClanci.json
  Obrada CSV: TVSviClanci.csv
Obrada direktorija: Region
  Obrada CSV: RegionSviClanciTreciDio.csv
  Obrada CSV: RegionSviClanciPrviDio.csv
  Obrada JSON: RegionSviClanciDrugiDio.json
  Obrada JSON: RegionSviClanciPrviDio.json
  Obrada CSV: RegionSviClanciDrugiDio.csv
  Obrada JSON: RegionSviClanciTreciDio.json
Obrada di

In [6]:
# Zipovanje i instaliranje foldera
# Kreiranje fajla /content/eval_zip.zip
import shutil

shutil.make_archive(
    base_name="/content/eval_zip",  # izlazni zip bez ekstenzije
    format="zip",
    root_dir="/content/drive/MyDrive/[F3]EvaluacijaTekstova"  # ulazni folder
)

'/content/eval_zip.zip'

In [7]:
# Preuzimanje zipa
from google.colab import files

files.download("/content/eval_zip.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>